In [1]:
import numpy as np
import pandas as pd
import time
import datetime
import gc
import random
import re
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler,random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import transformers
from transformers import BertForSequenceClassification, AdamW, BertConfig,BertTokenizer,get_linear_schedule_with_warmup
from bertTokenization import preprocess_tweets
from classifier import load_model
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import warnings
from torch.optim import Adam
from torch.optim import SGD
from torch.optim import ASGD
warnings.filterwarnings("ignore")

c:\Users\ss139\anaconda3\envs\transformer\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
# uncomment the below line to select the specific dataset topic for training the model
# ---Gun Control Dataset---
traindf = pd.read_csv("D:\ASProject\Transformer\imageArg\gun_control_train.csv")
valdf = pd.read_csv("D:\ASProject\Transformer\imageArg\gun_control_dev.csv")
#---abortion dataset---
#traindf = pd.read_csv(r"D:\ASProject\Transformer\imageArg\abortion_train.csv")
#valdf = pd.read_csv(r"D:\ASProject\Transformer\imageArg\abortion_dev.csv")
#---combined dataset---
#traindf_gc = pd.read_csv("D:\ASProject\Transformer\imageArg\gun_control_train.csv")
#valdf_gc = pd.read_csv("D:\ASProject\Transformer\imageArg\gun_control_dev.csv")
#traindf_ab = pd.read_csv(r"D:\ASProject\Transformer\imageArg\abortion_train.csv")
#valdf_ab = pd.read_csv(r"D:\ASProject\Transformer\imageArg\abortion_dev.csv")
#combined_train = pd.concat([traindf_gc, traindf_ab], ignore_index=True)
#combined_val = pd.concat([valdf_gc, valdf_ab], ignore_index=True)

In [4]:
columns_to_drop = ['tweet_url', 'persuasiveness', 'split']
traindf.drop(columns=columns_to_drop, inplace=True)
traindf['stance'] = traindf['stance'].replace({'oppose': 0, 'support': 1})
valdf.drop(columns=columns_to_drop, inplace=True)
valdf['stance'] = valdf['stance'].replace({'oppose': 0, 'support': 1})
# for combined data
#combined_train.drop(columns=columns_to_drop, inplace=True)
#combined_train['stance'] = combined_train['stance'].replace({'oppose': 0, 'support': 1})
#combined_val.drop(columns=columns_to_drop, inplace=True)
#combined_val['stance'] = combined_val['stance'].replace({'oppose': 0, 'support': 1})

In [5]:
# Clean the tweet_text column in the DataFrame
# Commenting out while training the model for the raw tweettext.
from dataCleaningScript import appriviation_converter
from dataCleaningScript import clean_text
traindf['tweet_text'] = traindf['tweet_text'].apply(appriviation_converter)
traindf['tweet_text'] = traindf['tweet_text'].apply(clean_text)
valdf['tweet_text'] = valdf['tweet_text'].apply(appriviation_converter)
valdf['tweet_text'] = valdf['tweet_text'].apply(clean_text)

# for combined data
#combined_train['tweet_text'] = combined_train['tweet_text'].apply(appriviation_converter)
#combined_train['tweet_text'] = combined_train['tweet_text'].apply(clean_text)
#combined_val['tweet_text'] = combined_val['tweet_text'].apply(appriviation_converter)
#combined_val['tweet_text'] = combined_val['tweet_text'].apply(clean_text)


In [6]:
train_tweets = traindf.tweet_text.values
trian_labels = traindf.stance.values
val_tweets = valdf.tweet_text.values
val_labels = valdf.stance.values
# for combined data
#train_tweets = combined_train.tweet_text.values
#trian_labels = combined_train.stance.values
#val_tweets = combined_val.tweet_text.values
#val_labels = combined_val.stance.values

In [7]:
# tokenized the tweet_texts and labels using the preprocess_tweets function.
# the function tokenized the text
input_ids_tr, attention_masks_tr, labels_tr = preprocess_tweets(train_tweets, trian_labels)
input_ids_v, attention_masks_v, labels_v = preprocess_tweets(val_tweets, val_labels)

In [8]:
# Combine the training inputs into a TensorDataset.
train_dataset = TensorDataset(input_ids_tr, attention_masks_tr, labels_tr)
val_dataset = TensorDataset(input_ids_v, attention_masks_v, labels_v)

# Calculate the number of samples to include in each set.
train_size = len(train_dataset)
val_size = len(val_dataset)

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  921 training samples
   98 validation samples


In [9]:
# The DataLoader needs to know our batch size for training, so we specify it here.
batch_size = 2

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly, This means that the training samples will be shuffled randomly in each epoch
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [10]:
# Loading the updated model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

Model is using device: cuda


In [11]:
# we are considering different "Optimizers" and "Learning Rate" as the hyperparameters.
# Adam optimizer that incorporates weight decay (L2 regularization) during optimization.
# It helps prevent overfitting by adding a penalty term to the loss function that encourages smaller weights.
    # 1. AdamW optimizer, and varying lr as 1e-5, 2e-5, 4e-5
#optimizer = AdamW(model.parameters(), lr = 2e-5, eps = 1e-8)
    # 2. Adam optimizer, and varying lr as 1e-5, 2e-5, 3e-5
optimizer = Adam(model.parameters(), lr = 3e-5, eps = 1e-8)
    # 3. SGD optimizer, and varying lr as 1e-2, 2e-2, 3e-2
#optimizer = SGD(model.parameters(), lr=3e-2)

In [12]:
# Fine tuning the model
# Number of training epochs.
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 8

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
# training step corresponds to one update of the model's parameters using a batch of training data.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
#The scheduler adjusts the learning rate during training, usually starting with a warm-up phase and then applying a linear decay schedule.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)


# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten() #comparing the index with the maximum value in preds to the corresponding index in labels
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss using the datetime.timedelta function.
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [13]:
from sklearn.metrics import f1_score
seed_val = 42
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# Initialize best_eval_accuracy to a very low value before starting the training loop.
best_eval_accuracy = 0
best_f1_score=0

# For each epoch...
for epoch_i in range(0, epochs):
    #               Training
    # Perform one full pass over the training set.
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    # Measure how long the training epoch takes.
    t0 = time.time() #track the start time of the epoch
    total_train_loss = 0
    model.train() #The model is set to training mode
    for step, batch in enumerate(train_dataloader):
        # Unpack this training batch from our dataloader. 
        # As we unpack the batch, we'll also copy each tensor to the device using the 
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # clear the gradients of the model parameters from the previous batch before computing the gradients for the current batch.
        # Clearing the gradients is essential to avoid unwanted gradient accumulation, especially when using mini-batches during training. 
        # No gradient command, result in incorrect gradient updates and potential convergence issues.
        optimizer.zero_grad()
        output = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)        
        loss = output.loss
        total_train_loss += loss.item() # Method in PyTorch that returns the loss value as a Python scalar.
        # Perform a backward pass to calculate the gradients.
        loss.backward()
        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are modified based on their gradients, the learning rate, etc.
        optimizer.step()
        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    print("  Average Training Loss: {0:.2f} || Training epoch took: {1}".format(avg_train_loss, training_time))
    #               Validation
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    t0 = time.time() 
    # Put the model in evaluation mode--the dropout layers behave differently during evaluation.
    model.eval()
    # Tracking variables 
    total_eval_accuracy = 0
    #best_eval_accuracy = 0
    total_eval_loss = 0
    total_eval_f1 = 0
    nb_eval_steps = 0
    # Evaluate data for one epoch
    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device) #The input tensors are moved to the specified device.
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        # Telling pytorch not to bother with constructing the compute graph during the forward pass, since this is only needed for backpropagation (training).
        with torch.no_grad():        
            output= model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = output.loss
        total_eval_loss += loss.item() # Method in PyTorch that returns the loss value as a Python scalar.
        # Move logits and labels to CPU if we are using GPU
        logits = output.logits
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        f1 = f1_score(label_ids, np.argmax(logits, axis=1), average='macro')
        total_eval_f1 += f1

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    avg_val_f1 = total_eval_f1 / len(validation_dataloader)
    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    if avg_val_f1 >= best_f1_score:
        torch.save(model, 'bert_model_gc')
        #torch.save(model, 'bert_model_ab')
        #torch.save(model, 'bert_model_combined')
        best_f1_score = avg_val_f1
    print(" Validation Accuracy: {0:.2f} || F1 Score: {1:.2f} || Validation took: {2}".format(avg_val_accuracy, avg_val_f1, validation_time))
    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. F1': avg_val_f1,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )
print("Training complete!")
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

======== Epoch 1 / 8 ========
  Average Training Loss: 0.68 || Training epoch took: 0:01:31
 Validation Accuracy: 0.91 || F1 Score: 0.88 || Validation took: 0:00:02
======== Epoch 2 / 8 ========
  Average Training Loss: 0.39 || Training epoch took: 0:01:29
 Validation Accuracy: 0.86 || F1 Score: 0.82 || Validation took: 0:00:02
======== Epoch 3 / 8 ========
  Average Training Loss: 0.15 || Training epoch took: 0:01:30
 Validation Accuracy: 0.92 || F1 Score: 0.90 || Validation took: 0:00:02
======== Epoch 4 / 8 ========
  Average Training Loss: 0.05 || Training epoch took: 0:01:28
 Validation Accuracy: 0.93 || F1 Score: 0.90 || Validation took: 0:00:02
======== Epoch 5 / 8 ========
  Average Training Loss: 0.02 || Training epoch took: 0:01:30
 Validation Accuracy: 0.95 || F1 Score: 0.93 || Validation took: 0:00:02
======== Epoch 6 / 8 ========
  Average Training Loss: 0.00 || Training epoch took: 0:01:30
 Validation Accuracy: 0.95 || F1 Score: 0.93 || Validation took: 0:00:02
======== E

In [16]:
model = torch.load('bert_model_gc')
#model = torch.load('bert_model_ab')
#model = torch.load('bert_model_combined')

In [17]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
# comment out the below line to test the gun control or abortion dataset
# -- Gun control test
df_test = pd.read_csv("gun_control_test.csv")
# -- Abortion test
#df_test = pd.read_csv("abortion_test.csv")
# -- Combined test
#gc_test = pd.read_csv("gun_control_test.csv")
#ab_test = pd.read_csv("abortion_test.csv")
#df_test = pd.concat([gc_test, ab_test], ignore_index=True)

columns_to_drop = ['tweet_url','split']
df_test.drop(columns=columns_to_drop, inplace=True)

# commented the text cleaning while testing on the raw dataset.
df_test['tweet_text'] = df_test['tweet_text'].apply(appriviation_converter)
df_test['tweet_text'] = df_test['tweet_text'].apply(clean_text)
test_tweets = df_test.tweet_text.values


# Call the preprocess_tweets function to tokenized the tweets and labels
test_input_ids, test_attention_masks = preprocess_tweets(test_tweets)

test_dataset = TensorDataset(test_input_ids, test_attention_masks)
test_dataloader = DataLoader(
            test_dataset, # The validation samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

predicted_labels = []
for batch in test_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        with torch.no_grad():        
            output= model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask)
            logits = output.logits
            logits = logits.detach().cpu().numpy()
            pred_flat = np.argmax(logits, axis=1).flatten()
            
            predicted_labels.extend(list(pred_flat))

df_output = pd.DataFrame()
df_output['tweet_id'] = df_test['tweet_id']
df_output['predicted_stance'] =predicted_labels
df_output.to_csv('result_gc.csv',index=False)
#df_output.to_csv('result_ab.csv',index=False) 
#df_output.to_csv('result_combined.csv',index=False) 

In [18]:
from prettytable import PrettyTable

# Sample data for the table - cleaned data
cleaned_data = [
    ['AdamW LR=1e-5', 0.92, 0.90],
    ['AdamW LR=2e-5', 0.93, 0.91],
    ['AdamW LR=4e-5', 0.93, 0.91],
    ['Adam LR=1e-5', 0.92, 0.90],
    ['Adam LR=2e-5', 0.94, 0.91],
    ['Adam LR=3e-5', 0.94, 0.92],
    ['SGD LR=1e-2', 0.87, 0.84],
    ['SGD LR=2e-2', 0.87, 0.85],
    ['SGD LR=3e-2', 0.90, 0.87],
]

# Sample data for the table - raw data
raw_data = [
    ['AdamW LR=1e-5', 0.91, 0.89],
    ['AdamW LR=2e-5', 0.93, 0.91],
    ['AdamW LR=4e-5', 0.91, 0.88],
    ['Adam LR=1e-5', 0.90, 0.88],
    ['Adam LR=2e-5', 0.89, 0.87],
    ['Adam LR=3e-5', 0.90, 0.87],
    ['SGD LR=1e-2', 0.90, 0.87],
    ['SGD LR=2e-2', 0.91, 0.86],
    ['SGD LR=3e-2', 0.91, 0.88],
]

# Create the PrettyTable and specify column names
table = PrettyTable()
table.field_names = ['Optimizer with LR', 'Cleaned Tweets Accuracy', 'Cleaned Tweets F1 Score', 'Raw Tweets Accuracy', 'Raw Tweets F1 Score']

# Add rows to the table
for cleaned_row, raw_row in zip(cleaned_data, raw_data):
    optimizer_lr_cleaned, acc_cleaned, f1_cleaned = cleaned_row
    optimizer_lr_raw, acc_raw, f1_raw = raw_row
    table.add_row([optimizer_lr_cleaned, acc_cleaned, f1_cleaned, acc_raw, f1_raw])

# Set the title of the table
table.title = "Results : Gun Control"

# Display the table
print(table)

+-------------------------------------------------------------------------------------------------------------------+
|                                               Results : Gun Control                                               |
+-------------------+-------------------------+-------------------------+---------------------+---------------------+
| Optimizer with LR | Cleaned Tweets Accuracy | Cleaned Tweets F1 Score | Raw Tweets Accuracy | Raw Tweets F1 Score |
+-------------------+-------------------------+-------------------------+---------------------+---------------------+
|   AdamW LR=1e-5   |           0.92          |           0.9           |         0.91        |         0.89        |
|   AdamW LR=2e-5   |           0.93          |           0.91          |         0.93        |         0.91        |
|   AdamW LR=4e-5   |           0.93          |           0.91          |         0.91        |         0.88        |
|    Adam LR=1e-5   |           0.92          |         

In [19]:
from prettytable import PrettyTable

# Sample data for the table - cleaned data
cleaned_data = [
    ['AdamW LR=1e-5', 0.91, 0.88],
    ['AdamW LR=2e-5', 0.93, 0.90],
    ['AdamW LR=3e-5', 0.91, 0.89],
    ['Adam LR=1e-5', 0.91, 0.89],
    ['Adam LR=2e-5', 0.92, 0.89],
    ['Adam LR=3e-5', 0.91, 0.88],
    ['SGD LR=1e-2', 0.89, 0.87],
    ['SGD LR=2e-2', 0.91, 0.89],
    ['SGD LR=3e-2', 0.94, 0.91],
]

# Sample data for the table - raw data
raw_data = [
    ['AdamW LR=1e-5', 0.92, 0.89],
    ['AdamW LR=2e-5', 0.92, 0.88],
    ['AdamW LR=4e-5', 0.92, 0.89],
    ['Adam LR=1e-5', 0.90, 0.82],
    ['Adam LR=2e-5', 0.93, 0.90],
    ['Adam LR=3e-5', 0.90, 0.87],
    ['SGD LR=1e-2', 0.90, 0.88],
    ['SGD LR=2e-2', 0.90, 0.88],
    ['SGD LR=3e-2', 0.92, 0.89],
]

# Create the PrettyTable and specify column names
table = PrettyTable()
table.field_names = ['Optimizer with LR', 'Cleaned Tweets Accuracy', 'Cleaned Tweets F1 Score', 'Raw Tweets Accuracy', 'Raw Tweets F1 Score']

# Add rows to the table
for cleaned_row, raw_row in zip(cleaned_data, raw_data):
    optimizer_lr_cleaned, acc_cleaned, f1_cleaned = cleaned_row
    optimizer_lr_raw, acc_raw, f1_raw = raw_row
    table.add_row([optimizer_lr_cleaned, acc_cleaned, f1_cleaned, acc_raw, f1_raw])

# Set the title of the table
table.title = "Results : Abortion"

# Display the table
print(table)

+-------------------------------------------------------------------------------------------------------------------+
|                                                 Results : Abortion                                                |
+-------------------+-------------------------+-------------------------+---------------------+---------------------+
| Optimizer with LR | Cleaned Tweets Accuracy | Cleaned Tweets F1 Score | Raw Tweets Accuracy | Raw Tweets F1 Score |
+-------------------+-------------------------+-------------------------+---------------------+---------------------+
|   AdamW LR=1e-5   |           0.91          |           0.88          |         0.92        |         0.89        |
|   AdamW LR=2e-5   |           0.93          |           0.9           |         0.92        |         0.88        |
|   AdamW LR=3e-5   |           0.91          |           0.89          |         0.92        |         0.89        |
|    Adam LR=1e-5   |           0.91          |         

In [20]:
# Results from the highest F1 score models of each topic are combined in the result.csv file.
result_gc = pd.read_csv("result_gc.csv")
result_ab = pd.read_csv("result_ab.csv")
result = result_gc.append(result_ab, ignore_index=True)

# -- results using the combined model
#result = pd.read_csv("result_combined.csv")

result.rename(columns={result.columns[1]: "stance"}, inplace=True)
result["stance"] = result["stance"].replace({0: "oppose", 1: "support"})

result.to_csv('team.bertmodel.TaskA.1.csv',index=False)

**Argumentative Stance (AS) Classification:**

We used BertForSequenceClassification model to classify the stance of the tweet text as either support or oppose.

**Implementation steps:**

*Tokenization and Data Loading*: Tokenize the tweets using the BERT tokenizer. It encodes the tweets by adding special tokens [CLS] and [SEP], pads/truncates them to the maximum token length, and creates attention masks. We created a function that Tokenize the tweets and returns three PyTorch tensors: input_ids, attention_masks, and labels, which represent the tokenized input, attention masks, and labels, respectively.  These tensors are combined into a PyTorch TensorDataset which is then split into training and validation set using the random_split function. Then we provided batches of size 2 to the TensorDataset using PyTorch DataLoader.

*Model Initialization*: Loads a pre-trained BERT model for sequence classification using the BertForSequenceClassification which is a 12 transformer layers BERT model. The base BERT model is extended by adding a classification layer on top. This additional layer is a linear transformation that takes the final hidden state of the BERT model and maps it to the 2 output labels considering the task as binary classification. The model by default, it uses a softmax activation function to produce probabilities for each class.

*Training*: Perform training for a specified number of epochs. For each epoch, the training data is passed through the model, and gradients are calculated to update the model's parameters using the optimizer. A learning rate scheduler is also used.

*Validation*: After each epoch, the model's performance is evaluated on the validation set. The accuracy and loss are computed, and the best model is saved based on validation accuracy.

*Evaluation*: Optimizer and learning rate were considered as hyperparameters. Three different optimizers (Adam, AdamW, and SGD) were used, each with various learning rates. The validation accuracy and F1 score were recorded for both the cleaned tweets and raw tweets of each topic, respectively.

The saved model with the highest F1 score for each topic was considered and tested on the respective test datasets separately. The "result.csv" file contains the combination of results for both the Gun Control and Abortion topics.